In [1]:
#import required libraries
from utils import aws # used to create aws session and load parquet 
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
import math

In [6]:
# Define trading strategy based on model predictions and trading parameters
def simple_trading_strategy(prediction, current_mid_price):
    if prediction > current_mid_price:
        return 'buy'  # Buy if the mid-price prediction is positive
    elif prediction < current_mid_price:
        return 'sell'  # Sell if the mid-price prediction is negative
    else:
        return 'hold'  # Hold if the mid-price prediction is neutral

In [7]:
# Implement trading simulator
class TradingSimulator:
    def __init__(self, historical_data, prediction_model, trading_strategy, initial_cash, feature_columns, prediction_type):
        """
        Initialize the TradingSimulator object with historical data, prediction model,
        trading strategy, and initial cash.

        Parameters:
        - historical_data: DataFrame containing historical market data
        - prediction_model: Model used to generate predictions
        - trading_strategy: Function defining the trading strategy
        - initial_cash: Initial cash available for trading
        - feature_columns: List of column headers used as features for the prediction
        - prediction_type: String that is either 'xgb' or 'normal' to define the prediction method used, 'normal' is the default
        """
        self.historical_data = historical_data
        self.prediction_model = prediction_model
        self.trading_strategy = trading_strategy
        self.feature_columns = feature_columns
        self.prediction_type = prediction_type
        self.portfolio = 0  # Initial portfolio value
        self.cash = initial_cash  # Initial cash
        self.profit_loss = []  # Store profit/loss at each time step
        self.trades = [] # Store trade history
        
    def run_prediction(self, features):
        """
        Use prediction model to generate predictions

        This method checks the prediction_type and calls the prediction_model predict function.
        
        Arguments:
        - features: A row of the dataframe with the values for the features used in model training
        """
        # 
        if self.prediction_type == 'xgb':
            features_df = pd.DataFrame(features).T
            features_df = features_df.apply(pd.to_numeric)
            # Convert features to DMatrix
            features_dmatrix = xgb.DMatrix(features_df)
            prediction = self.prediction_model.predict(features_dmatrix)
        elif self.prediction_type == 'normal':
            prediction = self.prediction_model.predict([features])[0]
        else:
            prediction = self.prediction_model.predict([features])[0]
        return prediction

    def run_simulation(self):
        """
        Run the trading simulation.

        This method iterates over historical data, generates predictions,
        applies trading strategy, and executes trades accordingly.
        """
        for index, row in self.historical_data.iterrows():
            # Extract features from historical data for prediction
            features = row[self.feature_columns]
            # Use prediction model to generate predictions
            prediction = self.run_prediction(features)

            # Get the current mid-price (replace 'current_mid_price' with the actual value)
            current_mid_price = row['Mid_Price_EMA_Short_z-score_normalised'] #TODO Replace with target

            # Implement trading strategy within the simulator
            trade_action = self.trading_strategy(prediction, current_mid_price)

            # Execute trade based on prediction
            # If the timestamp is outside the first 15 or last 30 minutes
            if row['Timestamp'] >= 90 and row['Timestamp'] < 28800:

                if trade_action == 'buy':
                    
                    if self.cash > 0:
                        # Invest all available cash in buying assets from level one in the orderbook
                        trade_price = row['Ask'][0][0] # lowest (level one) ask-price
                        max_quantity = math.floor(self.cash / trade_price) # max quantity of shares the model can afford to buy (must be an integer)
                        quantity_available = row['Ask'][0][1] # actual quantity of the lowest (level one) ask-price

                        if max_quantity < quantity_available:
                            # if the maximum number the model can afford is less than the quantity available buy the max it can afford 
                            quantity_to_buy = max_quantity
                        else:
                            # if the maximum number the model can afford is more than the quantity available buy the quantity available
                            quantity_to_buy = quantity_available

                        if index == 0:
                            # If it's the first row of historical data
                            # Buy action
                            self.portfolio += quantity_to_buy # Increase portfolio by bought quantity
                            self.cash = self.cash - (quantity_to_buy * trade_price) # Decrease cash by spent amount
                            # Record the trade in the trades list
                            self.trades.append([row['Date'], row['Timestamp'], ('buy', trade_price, quantity_to_buy)])
                        elif index > 0 and index - 1 < len(self.historical_data):
                            # If it's not the first row and there is more historical data
                            # Check if the trade price and quantity available are different from the previous row
                            if (trade_price != self.historical_data.iloc[index - 1]['Ask'][0][0] and quantity_available != self.historical_data.iloc[index - 1]['Ask'][0][1]):
                                # Buy action
                                self.portfolio += quantity_to_buy # Increase portfolio by bought quantity
                                self.cash = self.cash - (quantity_to_buy * trade_price) # Decrease cash by spent amount
                                # Record the trade in the trades list
                                self.trades.append([row['Date'], row['Timestamp'], ('buy', trade_price, quantity_to_buy)])
                            else: 
                                # Hold action if conditions are not met -- This prevents repeating the same trade over and over as the order book is not updated
                                self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                        else:
                            # Hold action if index is out of range
                            self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                        
                    else:
                        # Hold if no available cash
                        self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                        
                    
                elif trade_action == 'sell':
                    
                    if self.portfolio > 0:
                        # Invest all available shares to level one bid in the orderbook
                        trade_price = row['Bid'][0][0] # highest (level one) bid-price
                        quantity_available = row['Bid'][0][1] # actual quantity of the highest (level one) bid-price
                        max_quantity = self.portfolio # max quantity of shares the model has to sell (must be an integer)

                        if max_quantity < quantity_available:
                            # if the maximum number the model can sell is less than the quantity available sell the max it can
                            quantity_to_sell = max_quantity
                        else:
                            # if the maximum number the model can sell is more than the quantity available sell the quantity available
                            quantity_to_sell = quantity_available
                        
                        if index == 0:
                            # If it's the first row of historical data
                            # Sell action
                            self.portfolio -= quantity_to_sell # Decrease portfolio by sold quantity
                            self.cash += quantity_to_sell * trade_price  # Increase cash by earned amount
                            # Record the trade in the trades list
                            self.trades.append([row['Date'], row['Timestamp'], ('sell',trade_price, quantity_to_sell)])
                        elif index > 0 and index - 1 < len(self.historical_data):
                            # If it's not the first row and there is more historical data
                            # Check if the trade price and quantity available are different from the previous row
                            if (trade_price != self.historical_data.iloc[index - 1]['Bid'][0][0] and quantity_available != self.historical_data.iloc[index - 1]['Bid'][0][1]):
                                # Sell action
                                self.portfolio -= quantity_to_sell # Decrease portfolio by sold quantity
                                self.cash += quantity_to_sell * trade_price # Increase cash by earned amount
                                # Record the trade in the trades list
                                self.trades.append([row['Date'], row['Timestamp'], ('sell',trade_price, quantity_to_sell)])
                            else: 
                                # Hold action if conditions are not met -- This prevents repeating the same trade over and over as the order book is not updated
                                self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                        else: 
                            # Hold action if index is out of range    
                            self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])

                    else:
                        # Hold if no assets to sell
                        self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                        
                else:
                    # Hold if not a buy or sell signal
                    self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                
            # If the timestamp is between the last 30 mins and the last 15 minutes sell regardless of trade action, this is to close position.
            elif row['Timestamp'] >= 28800 and row['Timestamp'] < 29700:
            
                if self.portfolio > 0:
                    # If there are assets in the portfolio
                    trade_price = row['Bid'][0][0] # highest (level one) bid-price
                    quantity_available = row['Bid'][0][1] # actual quantity of the highest (level one) bid-price
                    max_quantity = self.portfolio # max quantity of shares the model has to sell (must be an integer)

                    # Calculate the quantity to sell based on available quantity and portfolio quantity
                    if max_quantity < quantity_available:
                        quantity_to_sell = max_quantity
                    else:
                        quantity_to_sell = quantity_available
                    
                    if index == 0:
                        # If it's the first row of historical data
                        self.portfolio -= quantity_to_sell
                        self.cash += quantity_to_sell * trade_price
                        # Record the trade in the trades list
                        self.trades.append([row['Date'], row['Timestamp'], ('sell',trade_price, quantity_to_sell)])
                    elif index > 0 and index - 1 < len(self.historical_data):
                        # If it's not the first row and there is more historical data
                        # Check if the trade price and quantity available are different from the previous row
                        if (trade_price != self.historical_data.iloc[index - 1]['Bid'][0][0] and quantity_available != self.historical_data.iloc[index - 1]['Bid'][0][1]):
                            self.portfolio -= quantity_to_sell
                            self.cash += quantity_to_sell * trade_price
                            # Record the trade in the trades list
                            self.trades.append([row['Date'], row['Timestamp'], ('sell',trade_price, quantity_to_sell)])
                        else: 
                            # Hold action if conditions are not met
                            self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                    else: 
                        # Hold action if index is out of range
                        self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
                    
                else:
                    # Hold if no assets to sell
                    self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])
            
            # If the timestamp is in the first 15 minutes or the last 15 minutes hold regardless of trade action, attempt to minimise impact of market open and market close conditions.
            else:
                # If the timestamp is in the first 15 minutes or the last 15 minutes hold regardless of trade action, attempt to minimise impact of market open and market close conditions.
                self.trades.append([row['Date'], row['Timestamp'], ('hold', None, None)])

        return self.trades
            
    def get_performance_metrics(self):
        # Return performance metrics such as profit/loss and portfolio
        return self.cash, self.portfolio

In [8]:
# Define data - must contain the features used in model training and the Bid/Ask columns
df = pd.read_parquet('data/preprocessed_feature_set.parquet')

# Preprocess the data to extract the target column
horizon = 20 # The horizon is how far in the future the Mid-Proce is being predicted
df = df.copy()
df['Target'] = df['Mid_Price_EMA_Short_z-score_normalised'].shift(-horizon)

# Drop rows where all values in the 'Target' column are NaN
X_test = df.dropna(subset=['Target'], how='all')

In [ ]:
# Import the initial trial XGBoost model
import xgboost as xgb

xgboost_model = xgb.Booster()
xgboost_model.load_model('models/full-features.xgb')

In [9]:
# Define a list of features to be used by the model for prediction
feature_columns = [
    "Mid_Price_Raw",
    "Mid_Price_EMA_Raw",
    "Unnamed: 0_z-score_normalised",
    "Mid_Price_z-score_normalised",
    "Total_Order_Volume_z-score_normalised",
    "OBV_z-score_normalised",
    "Total_Volume_Imbalance_z-score_normalised",
    "Order_Imbalance_z-score_normalised",
    "Weighted_Mid_Price_z-score_normalised",
    "Bid_Ask_Spread_z-score_normalised",
    "Level_1_Bid_Price_z-score_normalised",
    "Level_1_Ask_Price_z-score_normalised",
    "Level_1_Bid_Quantity_z-score_normalised",
    "Level_1_Ask_Quantity_z-score_normalised",
    "Level_1_Order_Imbalance_z-score_normalised",
    "RSI_z-score_normalised",
    "Stochastic_RSI_z-score_normalised",
    "Awesome_Oscillator_z-score_normalised",
    "Accelerator_Oscillator_z-score_normalised",
    "MACD_z-score_normalised",
    "MACD_Signal_z-score_normalised",
    "Hull_MA_z-score_normalised",
    "Keltner_Channel_Middle_z-score_normalised",
    "Keltner_Channel_Upper_z-score_normalised",
    "Keltner_Channel_Lower_z-score_normalised",
    "SMA_20_z-score_normalised",
    "DPO_z-score_normalised",
    "Upper_BB_z-score_normalised",
    "Lower_BB_z-score_normalised",
    "Log_Returns_z-score_normalised",
    "Realised_Semi_Variance_z-score_normalised",
    "Squared_Log_Returns_z-score_normalised",
    "Realised_Volatility_z-score_normalised",
    "Abs_Log_Returns_z-score_normalised",
    "Realised_Bipower_Variation_z-score_normalised",
    "Total_Quadratic_Variation_z-score_normalised",
    "Jump_Variation_z-score_normalised",
    "Smoothed_Mid_Price_z-score_normalised",
    "Mid_Price_EMA_Short_z-score_normalised"
]

In [15]:
# Run simulation using the initial train XGBoost model
xg_simulator = TradingSimulator(X_test, xgboost_model, simple_trading_strategy, initial_cash=100000, feature_columns=feature_columns, prediction_type='xgb')
trades = xg_simulator.run_simulation()
trades

[['2025-05-06', 1.984, ('hold', None, None)],
 ['2025-05-06', 2.697, ('hold', None, None)],
 ['2025-05-06', 2.759, ('hold', None, None)],
 ['2025-05-06', 3.286, ('hold', None, None)],
 ['2025-05-06', 3.596, ('hold', None, None)],
 ['2025-05-06', 3.72, ('hold', None, None)],
 ['2025-05-06', 3.875, ('hold', None, None)],
 ['2025-05-06', 3.937, ('hold', None, None)],
 ['2025-05-06', 4.03, ('hold', None, None)],
 ['2025-05-06', 4.309, ('hold', None, None)],
 ['2025-05-06', 4.619, ('hold', None, None)],
 ['2025-05-06', 4.867, ('hold', None, None)],
 ['2025-05-06', 4.898, ('hold', None, None)],
 ['2025-05-06', 4.929, ('hold', None, None)],
 ['2025-05-06', 5.053, ('hold', None, None)],
 ['2025-05-06', 5.084, ('hold', None, None)],
 ['2025-05-06', 5.208, ('hold', None, None)],
 ['2025-05-06', 5.332, ('hold', None, None)],
 ['2025-05-06', 5.363, ('hold', None, None)],
 ['2025-05-06', 5.456, ('hold', None, None)],
 ['2025-05-06', 5.58, ('hold', None, None)],
 ['2025-05-06', 5.735, ('hold', None,

In [16]:
# Get performance metrics from the XGBoost Trial simulation
xg_performance_metrics = xg_simulator.get_performance_metrics()
print(xg_performance_metrics)

(183, 0)


## Things To Note

- How do we deal with the fact that every order we make will impact the data and therefore we need to remove the bids/asks from the historical data when they are taken by the trade. The issue with the current implementation of the trading simulator is that when the Market order is applied the corresponding bid/ask remains in the dataset.
    - To combat this I have only allowed the model to make a single trade with a corresponding bid/ask this allows the simulator to act as a paper trading simulation where the trades made have no impact on the data, however it limits the model to make the trade only once rather than repeating the trade because the limit order is not updated.
- To run the simulator the relevant feature columns will need to be defined and the new model imported. Before running the model check the value being predicted is the same value as that defined in "current_mid_price" but shifted by the defined horizon.